<a href="https://colab.research.google.com/github/sumanmichael/variational-quantum-classifier/blob/master/WrapperVQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing a few dependencies
!pip install --upgrade seaborn==0.10.1
!pip install --upgrade scikit-learn==0.23.1
!pip install --upgrade matplotlib==3.2.0
!pip install --upgrade pandas==1.0.4
!pip install --upgrade qiskit==0.19.6 
!pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [ ]:
from qiskit import *
def wrapper(custom_data_map_func,MAX_ITER=500,VAR_DEPTH=2,FEATURE_DEPTH=2,TRAIN_SIZE=200,TEST_SIZE=10,GRADE_SIZE=2000):
    from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
    from qiskit import Aer

    
    import numpy as np
    from qiskit.visualization import plot_bloch_multivector, plot_histogram
    %matplotlib inline
    import matplotlib.pyplot as plt

    import time
    from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
    from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
    from qiskit.aqua import QuantumInstance
    from qiskit.aqua.algorithms import VQC
    from qiskit.aqua.components.optimizers import COBYLA

    # ---------------

    data_path='./dataset/'
    data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

    # extracting the first column which contains the labels
    data_labels = data[:, :1].reshape(data.shape[0],)
    # extracting all the columns but the first which are our features
    data_features = data[:, 1:]

    # -------------------

    four_features = []
    nine_features = []
    for i in range(len(data_features)):
        if(data_labels[i]==4):
            four_features.append(data_features[i])
        elif(data_labels[i]==9):
            nine_features.append(data_features[i])

    train_size = TRAIN_SIZE
    test_size = TEST_SIZE


    four_train = four_features[:train_size]
    nine_train = nine_features[:train_size]
    four_test = four_features[train_size + 1:train_size + test_size + 1]
    nine_test = nine_features[train_size + 1:train_size + test_size + 1]

    four_train = np.array(four_train)
    nine_train = np.array(nine_train)


    training_input = {'A':four_train, 'B':nine_train}
    testing_input =  {'A':four_test, 'B':nine_test}

    #-----------------------

    # import libraries that are used in the function below.
    from qiskit import QuantumCircuit
    from qiskit.circuit import ParameterVector
    from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
        
    ### WRITE YOUR CODE BETWEEN THESE LINES - END

    def feature_map(): 
        # BUILD FEATURE MAP HERE - START
        
        # import required qiskit libraries if additional libraries are required
        feature_dim = 3
        # build the feature map
        # feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=FEATURE_DEPTH, data_map_func=custom_data_map_func)
        feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=FEATURE_DEPTH, paulis = ['Z','X','Y','XY','ZZ'])
        # BUILD FEATURE MAP HERE - END
        
        #return the feature map which is either a FeatureMap or QuantumCircuit object
        return feature_map

    feature_map().draw()
    #-------------------------


    # import libraries that are used in the function below.
    from qiskit import QuantumCircuit
    from qiskit.circuit import ParameterVector
    from qiskit.circuit.library import  RealAmplitudes, EfficientSU2
        
    ### WRITE YOUR CODE BETWEEN THESE LINES - END

    def variational_circuit():
        # BUILD VARIATIONAL CIRCUIT HERE - START
        
        # import required qiskit libraries if additional libraries are required
        from qiskit.circuit.library import ExcitationPreserving
        feature_dim = 3
        # build the variational circuit
        var_circuit = EfficientSU2(feature_dim, reps=VAR_DEPTH)

        # BUILD VARIATIONAL CIRCUIT HERE - END
        
        # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
        return var_circuit

    #-------------------------

    def classical_optimizer():
        # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
        
        # import the required clasical optimizer from qiskit.aqua.optimizers
        from qiskit.aqua.components.optimizers import COBYLA,ADAM,SLSQP
        # create an optimizer object
        cls_opt = COBYLA(maxiter=MAX_ITER, tol=0.001)
        
        # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
        return cls_opt

    #------------------------
    evals=[]
    def call_back_vqc(eval_count, var_params, eval_val, index):
        print("eval_count: {}".format(eval_count))
        print("var_params: {}".format(var_params))
        print("eval_val: {}".format(eval_val))
        print("index: {}".format(index))
        evals.append(eval_val)

    # -------------------------

    # a fixed seed so that we get the same answer when the same input is given. 
    seed = 10598

    from qiskit.providers.aer import QasmSimulator
    # setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
    # found to perform better than most. Feel free to play around with different backend options.
    backend = Aer.get_backend('qasm_simulator')
    backend_options = {"method": "statevector"}

    # creating a quantum instance using the backend and backend options taken before
    quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                    backend_options=backend_options)

    # creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
    # testing_dataset as defined in your program.
    vqc = VQC(optimizer=classical_optimizer(), 
            feature_map=feature_map(), 
            var_form=variational_circuit(), 
            callback=call_back_vqc, 
            training_dataset=training_input,     # training_input must be initialized with your training dataset
            test_dataset=testing_input) 


    # ---------------------

    start = time.process_time()

    result = vqc.run(quantum_instance)

    print("time taken: ")
    print(time.process_time() - start)

    print("testing success ratio: {}".format(result['testing_accuracy']))

    # --------------------------

    import numpy as np
        
    ### WRITE YOUR CODE BETWEEN THESE LINES - END
    
    def return_optimal_params():
        # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
        return vqc.optimal_params

    # -------------------------

    #imports required for the grading function 
    from qiskit.aqua import QuantumInstance
    from qiskit.aqua.algorithms import VQC
    from qiskit.aqua.components.feature_maps import FeatureMap
    from qiskit.aqua.components.variational_forms import VariationalForm
    import numpy as np

    # -----------------------

    def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
        seed = 10598
        model_accuracy = None 
        circuit_cost=None 
        ans = None
        unrolled_circuit = None
        result_msg=''
        data_dim = np.array(test_data).shape[1]
        dataset_size = np.array(test_data).shape[0]
        dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
        
        # converting 4's to 0's and 9's to 1's for checking 
        test_labels_transformed = np.where(test_labels==4, 0., 1.)
        max_qubit_count = 6
        max_circuit_cost = 2000
        
        # Section 1
        if feature_map is None:
            result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
        elif variational_form is None: 
            result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
        elif optimal_params is None: 
            result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
        elif test_data is None: 
            result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
        elif test_labels is None: 
            result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
        elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
            result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                        type(feature_map) if verbose else ''
        elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
            result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                        type(variational_form) if verbose else ''
        elif not isinstance(test_data, np.ndarray):
            result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                        type(test_data) if verbose else ''
        elif not isinstance(test_labels, np.ndarray):
            result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                        type(test_labels) if verbose else ''
        elif not isinstance(optimal_params, np.ndarray):
            result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                        type(optimal_params) if verbose else ''
        elif not dataset_size == test_labels_transformed.shape[0]:
            result_msg += 'Dataset size and label array size must be equal'
        # Section 2
        else:
            
            # setting up COBYLA optimizer as a dummy optimizer
            from qiskit.aqua.components.optimizers import COBYLA
            dummy_optimizer = COBYLA()
    
            # setting up the backend and creating a quantum instance
            backend = Aer.get_backend('qasm_simulator')
            backend_options = {"method": "statevector"}
            quantum_instance = QuantumInstance(backend, 
                                            shots=2000, 
                                            seed_simulator=seed, 
                                            seed_transpiler=seed, 
                                            backend_options=backend_options)
    
            # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
            vqc = VQC(optimizer=dummy_optimizer, 
                    feature_map=feature_map, 
                    var_form=variational_form, 
                    training_dataset=dummy_training_dataset)
            
            from qiskit.transpiler import PassManager
            from qiskit.transpiler.passes import Unroller
            pass_ = Unroller(['u3', 'cx'])
            pm = PassManager(pass_)
            # construct circuit with first datapoint
            circuit = vqc.construct_circuit(data[0], optimal_params)
            unrolled_circuit = pm.run(circuit)
            gates = unrolled_circuit.count_ops()
            if 'u3' in gates: 
                circuit_cost = gates['u3']
            if 'cx' in gates: 
                circuit_cost+= 10*gates['cx']
            
            if circuit.num_qubits > max_qubit_count:
                result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
            elif circuit_cost > max_circuit_cost:
                result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
            else: 
                
                ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
                model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])
    
                result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
                result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
                
        return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

    #-----------------------------

    grading_dataset_size=GRADE_SIZE    # this value is not per digit but in total
    grading_features = data_features[-grading_dataset_size:]
    grading_labels = data_labels[-grading_dataset_size:]


    # --------------------------

    start = time.process_time()
    
    accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                    test_labels=grading_labels, 
                                                                    feature_map=feature_map(), 
                                                                    variational_form=variational_circuit(), 
                                                                    optimal_params=return_optimal_params())
    
    print("time taken: {} seconds".format(time.process_time() - start))
    print(result_msg)
    print(repr(return_optimal_params()))
    print(evals[-1])
    return evals

In [ ]:
import numpy as np
import functools
import matplotlib.pyplot as plt

def custom_data_map_func(x):
    if len(x) == 1:
      coeff = x[0]
    else:
      coeff = functools.reduce(lambda m, n: m * n, np.pi-x)
    return coeff

evals = wrapper(custom_data_map_func,MAX_ITER=500,VAR_DEPTH=6,FEATURE_DEPTH=2,TRAIN_SIZE=2000,TEST_SIZE=50)
plt.plot(range(len(evals)),evals)

OSError: ignored